In [1]:
import os
import torch
import torchvision.transforms as tt
import time
import datetime
from torch.utils.data import Dataset, DataLoader, random_split
from numpy import array, random, genfromtxt, shape, zeros, ones, amin, amax

In [2]:
# Device Check
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#device = torch.device('cpu')
print(device)

cuda


In [3]:
def nonZeroMat(inpTensor):
    nRow, nCol   = torch.Tensor.size(inpTensor)
    origMatArray = torch.zeros(nRow, nCol)
    for i in range(nRow):
        for j in range(nCol):
            val = inpTensor[i,j]
            if val != 0:
                origMatArray[i,j] = 1
            else:
                pass
    return origMatArray

In [4]:
def strainSq(ex, ey, exy, nu):
    
    CH1 = (1-nu)*ex**2
    CH2 = (1-nu)*ey**2
    CH3 = 2*ex*ey
    CH4 = 2*(1-2*nu)*exy**2
    
    return CH1, CH2, CH3, CH4

def strainCalc(exTensor, eyTensor, exyTensor):
    outTensor = 2e11*exTensor ** 2 + 2e11*eyTensor ** 2 + 2e11/(2+2*0.3)* exyTensor ** 2
    return outTensor

In [5]:
def dataSegment_lv2(matDenArray):
    nRow, nCol = shape(matDenArray)
    segMatDen  = zeros((nRow, nCol))
    
    for i in range(nRow):
        for j in range(nCol):
            val = matDenArray[i,j]
            if val <= 0.5:
                ind = int(0)
            else:
                ind = int(1)
            segMatDen[i,j] = ind
    
    segMatDenTensor = torch.LongTensor(segMatDen)
    return segMatDenTensor

In [6]:
# DATA LOADER CLASS
class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.dataTemp    = torch.FloatTensor(data)
        self.targetsTemp = torch.LongTensor(targets)
        self.data        = self.dataTemp.cuda()
        self.targets     = self.targetsTemp.cuda()
        
        #self.targets.requires_grad_(True)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y

    def __len__(self):
        return len(self.data)

# TORCH ROTATION FUNCTION
def torchRotFunc(data, index):
    filpData = torch.fliplr(data)
    if index == 0:
        newData = data[:]
    elif index == 1:
        newData = torch.rot90(data)
    elif index == 2:
        newData = torch.rot90(torch.rot90(data))
    elif index == 3:
        newData = torch.rot90(torch.rot90(torch.rot90(data)))
    elif index == 4:
        newData = torch.fliplr(data)
    elif index == 5:
        newData = torch.rot90(filpData)
    elif index == 6:
        newData = torch.rot90(torch.rot90(filpData))
    elif index == 7:
        newData = torch.rot90(torch.rot90(torch.rot90(filpData)))
    return newData

# NORMALIZATION FUNCTION
def normalTensor(inpTensor, limRange):
    '''
    To scale the variable, x, into a range [a,b] as:
    x_norm = (b-a)*(x-x_min)/(x_max-x_min) + a
    * Nom material region --> Keep using Zero
    '''
    a              = limRange[0]
    b              = limRange[1]
    
    minVal         = inpTensor.min()
    maxVal         = inpTensor.max()
    numRow, numCol = inpTensor.size()
    outTensor      = torch.zeros(numRow, numCol)
    
    for i in range(numRow):
        for j in range(numCol):
            testVal = inpTensor[i,j]
            if testVal != 0:
                outTensor[i,j] = (b-a)*(testVal-minVal)/(maxVal-minVal) + a
            else:
                pass
    
    return outTensor

In [7]:
# DATA EXNTESION USING FLIPLR
deletOption      = False
numData          = 25

numInputChannel  = 5
numInputRow      = 80
numInputColumn   = 80

numOutputChannel = 1
numOutputRow     = 80
numOutputColumn  = 80

# DATA EXNTESION USING FLIPLR
limRange         = [10, 100]

# Initialize Output Dataset
inputData    = torch.zeros((numData*4, numInputChannel, numInputRow, numInputColumn))
outputData   = torch.zeros((numData*4, numOutputRow, numOutputColumn), dtype=torch.int64)

# Data Allocation
for i in range(numData):
    fileLoc     = '.\\ppData\\'
    
    exFileName     = fileLoc + 'ex_80x80_' + str(i) + '.pt'
    eyFileName     = fileLoc + 'ey_80x80_' + str(i) + '.pt'
    exyFileName    = fileLoc + 'exy_80x80_' + str(i) + '.pt'
    matDenFileName = fileLoc + 'matDen_80x80_' + str(i) + '.pt'
    
    ex     = torch.load(exFileName)
    ey     = torch.load(eyFileName)
    exy    = torch.load(exyFileName)
    matDen = torch.load(matDenFileName)
    
    dtat_1_SE, dtat_2_SE, dtat_3_SE, data_4_SE = strainSq(ex, ey, exy, 0.3)
    
    data_1      = normalTensor(dtat_1_SE, limRange)
    data_2      = normalTensor(dtat_2_SE, limRange)
    data_3      = normalTensor(dtat_3_SE, limRange)
    data_4      = normalTensor(data_4_SE, limRange)
    data_0      = nonZeroMat(data_1)
    data_6      = dataSegment_lv2(matDen)

    
    # ORIGINAL DATA
    inputData[i][0]  = data_0
    inputData[i][1]  = data_1
    inputData[i][2]  = data_2
    inputData[i][3]  = data_3
    inputData[i][4]  = data_4
    
    outputData[i]    = data_6
    
    # DATA AUGMENTATION
    inputData[i+numData][0]  = torch.flipud(data_0)
    inputData[i+numData][1]  = torch.flipud(data_1)
    inputData[i+numData][2]  = torch.flipud(data_2)
    inputData[i+numData][3]  = torch.flipud(data_3)
    inputData[i+numData][4]  = torch.flipud(data_4)
    
    outputData[i+numData]    = torch.flipud(data_6)
    
    inputData[i+numData*2][0]  = torch.fliplr(data_0)
    inputData[i+numData*2][1]  = torch.fliplr(data_1)
    inputData[i+numData*2][2]  = torch.fliplr(data_2)
    inputData[i+numData*2][3]  = torch.fliplr(data_3)
    inputData[i+numData*2][4]  = torch.fliplr(data_4)
    
    outputData[i+numData*2]    = torch.fliplr(data_6)
    
    inputData[i+numData*3][0]  = torch.fliplr(torch.flipud(data_0))
    inputData[i+numData*3][1]  = torch.fliplr(torch.flipud(data_1))
    inputData[i+numData*3][2]  = torch.fliplr(torch.flipud(data_2))
    inputData[i+numData*3][3]  = torch.fliplr(torch.flipud(data_3))
    inputData[i+numData*3][4]  = torch.fliplr(torch.flipud(data_4))
    
    outputData[i+numData*3]    = torch.fliplr(torch.flipud(data_6))
    
    if (i+1)%50 ==0:
        now = datetime.datetime.now()
        date_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print(f'Progress {i+1}/{numData} // Time: {date_time}')        
        

In [9]:
# DEVELOP TRAIN & TEST DATA SETS
'''
Typically, # of Train : # of Test = 8:2
'''
trainDataset = MyDataset(inputData, outputData)

numTrainData = 50
numValidData = 50
batchSize    = 50

train_data, test_data = random_split(trainDataset, (numTrainData,numValidData))
train_dl = DataLoader(train_data, batch_size=batchSize)
test_dl  = DataLoader(test_data,  batch_size=batchSize)

# Data Loader Save
torch.save(train_dl, 'TRAIN_DL.pth')
torch.save(test_dl,  'TEST_DL.pth')